In [1]:
import pandas as pd
import numpy as np
import statistics as stat
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("white")
import statistics as stat

In [2]:
data = pd.read_csv('scdb-20rfq-60cap-metrics.csv', names = ['Event', 'Time', 'Tx_type', 'Tx_id', 'Accept_id', 'time'])

In [3]:
grouped_data = data.groupby('Tx_id')
tx_id = list(grouped_data.groups.keys())
len(tx_id)

1538

In [4]:
#new data frame to get the count of each transaction type
data_unique_ids = data.drop_duplicates(subset = ['Tx_id'])

bid_count = 0
bid_ids = []
for i in range(0, len(data_unique_ids)):
    if data_unique_ids.iloc[i]['Tx_type'] == 'BID':
        bid_ids.append(data_unique_ids.iloc[i]['Tx_id'])
        bid_count += 1   


print(f"#BIDs =  {bid_count}")

#BIDs =  499


In [5]:
df_test = grouped_data.get_group(bid_ids[8])
df_test

,Event,Time,Tx_type,Tx_id,Accept_id,time
3186,received_tx,109,BID,1572925618f77a02467a703d40545326ab278a55e0b825...,None,1.669849e+09
3187,before_tendermint,122,BID,1572925618f77a02467a703d40545326ab278a55e0b825...,None,1.669849e+09
3188,check_tx,135,BID,1572925618f77a02467a703d40545326ab278a55e0b825...,None,1.669849e+09
3189,deliver_tx,152,BID,1572925618f77a02467a703d40545326ab278a55e0b825...,None,1.669849e+09
3190,end_block,154,BID,1572925618f77a02467a703d40545326ab278a55e0b825...,None,1.669849e+09
3191,commit_tx,196,BID,1572925618f77a02467a703d40545326ab278a55e0b825...,None,1.669849e+09


In [6]:
time_all_committed_bids = {}
time_passed_bids = []
validated_time = []
accepted_time = []
committed_time = []

bid_underfit = []
bid_underfit_dict = {}

bid_overfit = []
bid_overfit_dict = {}

for i in range(0, len(tx_id)):
    temp_list = []
    if "BID" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
        temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
        if len(temp_list) == 6 and grouped_data.get_group(tx_id[i])["Event"].to_list()[5] == "commit_tx":
            
            actual_time = max(temp_list) - min(temp_list)
            time_all_committed_bids[tx_id[i]] = actual_time
            time_passed_bids.append(actual_time)
            
            df = grouped_data.get_group(tx_id[i])
            received_time = df[df['Event'] == 'received_tx']['Time'].to_list()[0]
            before_tendermint_time = df[df['Event'] == 'before_tendermint']['Time'].to_list()[0]
            
            validated = before_tendermint_time - received_time
            validated_time.append(validated)
            
            check_time = df[df['Event'] == 'check_tx']['Time'].to_list()[0]
            accepted = check_time - received_time
            accepted_time.append(accepted)
            
            commit_time =  df[df['Event'] == 'commit_tx']['Time'].to_list()[0]
            committed = commit_time - received_time
            committed_time.append(committed)
        elif len(temp_list) < 6:
            bid_underfit.append(temp_list)
            bid_underfit_dict[tx_id[i]] = temp_list
        elif 7 >= len(temp_list) > 6 :
            df_check = grouped_data.get_group(tx_id[i])
            if len(df_check[df_check['Event'] == 'check_tx']['Time'].to_list()) > 1:
                check_time = max(df_check[df_check['Event'] == 'check_tx']['Time'].to_list())
            else:
                check_time = df_check[df_check['Event'] == 'check_tx']['Time'].to_list()[0] 
            
            actual_time = max(temp_list) - min(temp_list)
            time_all_committed_bids[tx_id[i]] = actual_time
            #time_passed_bids.append(actual_time)
            
            df = grouped_data.get_group(tx_id[i])
            received_time = df[df['Event'] == 'received_tx']['Time'].to_list()[0]
            before_tendermint_time = df[df['Event'] == 'before_tendermint']['Time'].to_list()[0]
            
            validated = before_tendermint_time - received_time
            validated_time.append(validated)
           
            accepted = check_time - received_time
            accepted_time.append(accepted)
            
            commit_time =  df[df['Event'] == 'commit_tx']['Time'].to_list()[0]
            committed = commit_time - received_time
            committed_time.append(committed)
            
            bid_overfit.append(actual_time)
            bid_overfit_dict[tx_id[i]] = temp_list

In [7]:
df = grouped_data.get_group("3066cad3ab9009c84499c002f3b150864fc0b5af46bd4cb1ab5407ab2396a336")
df[df['Event'] == 'received_tx']['Time'].to_list()[0]

KeyError: '3066cad3ab9009c84499c002f3b150864fc0b5af46bd4cb1ab5407ab2396a336'

In [ ]:
df[df['Event'] == 'received_tx']

In [8]:
bid_overfit_dict

{'00eae556b6f3fccec3885b4e1e7e682488cf8b9321430aba36b6d03b0c891aa8': [113,
  124,
  144,
  891,
  992,
  994,
  1040],
 '0bb7e08feca3d2d01f2761e4dd6ad2f3848392c2a5d6d54157fba1c0803fcd58': [108,
  118,
  144,
  346,
  405,
  407,
  452],
 '12b610514016492f45f8f74e3e6bbd91e070f4d17f07de59d5186d665ad15b22': [109,
  124,
  145,
  313,
  428,
  431,
  440],
 '13df832129e15f77ce288c44da2f9f40f529035379a915d60dd8e0351ee650d1': [107,
  118,
  142,
  341,
  401,
  402,
  445],
 '157b09d6b06b11de05b194ed6e1dd1210776a548c2a19c6f2627f5ba1a32cd5a': [108,
  119,
  142,
  363,
  422,
  424,
  466],
 '24d4c7adc66e01cba4519a987062ced3d6247df84c949f0051bb73d144973c1c': [109,
  119,
  276,
  442,
  510,
  511,
  545],
 '3c590795b04403ee6a12ab3379fb49739e3bc2270675d4445053f5b8bfeae668': [108,
  118,
  198,
  464,
  621,
  697,
  706],
 '43995f2e5d9ba8a33b5bc85fb0e4fed32eba27332a1ff89d9dd0e56219829e30': [108,
  119,
  208,
  362,
  421,
  423,
  465],
 '585b17fcf83991aada1b648b471a91f39ec096a0f3ee5793a3012

In [9]:
len(time_passed_bids)

483

In [10]:
len(bid_overfit)

16

In [11]:
stat.mean(time_passed_bids)

104.48447204968944

In [12]:
stat.mean(bid_overfit)

455.1875

In [13]:
# Analysis of CREATE Transaction
bid_mean = "{:.2f}".format(stat.mean(time_passed_bids))
bid_median = stat.median(time_passed_bids)
bid_max = max(time_passed_bids)
bid_min = min(time_passed_bids)
bid_std_dev = "{:.2f}".format(stat.stdev(time_passed_bids))

print(bid_mean)
print(bid_median)
print(bid_max)
print(bid_min)
print(bid_std_dev)
print(f"total bids = {len(time_passed_bids)}")

bid_dict = {
               'min': bid_min,
               'max': bid_max,
               'average': bid_mean,
               'median' : bid_median,
               'standard_deviation' : bid_std_dev,
               '# of txs' : len(time_passed_bids)}

104.48
88
753
77
55.26
total bids = 483


In [14]:
results = pd.DataFrame(bid_dict, index = ['BID'])
results.to_csv('bid_analysis.csv', sep=',', encoding='utf-8')
results

,min,max,average,median,standard_deviation,# of txs
BID,77,753,104.48,88,55.26,483
